# Toronto Neighborhoods Assignment (3)

### Heriberto Encinas López

## 1.- Load and Explore Dataset

In this section we cluster the data from Toronto neighbourhoods and discuss the insights we obtained from it.

First we load all the libraries that we will use in this Jupyter Notebook to analyze Toronto Neighbourhood data.

In [1]:
#first we import the libraries we need
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
from pandas.io.json import json_normalize
import json

And we we load the data we obtained afte running the Toronto Data 2 jupyter notebook.

In [2]:
#we load our data
data = pd.read_csv('final_dataset.csv')
data.head()

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, West Hill, Morningside  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

The data is already sorted and already includes latitude and longitude values.

#### Use geopy to get Toronto's latitude and longitude values.

In [3]:
#get the coordinates of Toronto
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Select boroughs that contain the word 'Toronto'

Here we use a small trick to drop all the instances that do not include 'Toronto' in their Borough name.

In [4]:
indeces = data['Borough'].str.contains('Toronto')
torontodata = data[indeces]

#### Create a map of Toronto with neighbourhoods superimposed on top.

Here we build a map to visualize the remaining data. This only includes instances that contain the word 'Toronto' inside their Borough name.

In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontodata['Latitude'], torontodata['Longitude'], torontodata['Borough'], torontodata['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Selecting neighbourhoods in Central Toronto

I decided to focus on the Central Toronto borough. I expect to find a significant number of venues since this is the "Central" area and therefore it should be an important area.

In [6]:
centoronto_data = torontodata[torontodata['Borough']=='Central Toronto'].reset_index(drop=True)
centoronto_data.head(100)

Postal Code          Borough  \
0         M4N  Central Toronto   
1         M4P  Central Toronto   
2         M4R  Central Toronto   
3         M4S  Central Toronto   
4         M4T  Central Toronto   
5         M4V  Central Toronto   
6         M5N  Central Toronto   
7         M5P  Central Toronto   
8         M5R  Central Toronto   

                                       Neighbourhood   Latitude  Longitude  
0                                      Lawrence Park  43.728020 -79.388790  
1                                   Davisville North  43.712751 -79.390197  
2                                 North Toronto West  43.715383 -79.405678  
3                                         Davisville  43.704324 -79.388790  
4                        Summerhill East, Moore Park  43.689574 -79.383160  
5  Forest Hill SE, Deer Park, Summerhill West, So...  43.686412 -79.400049  
6                                           Roselawn  43.711695 -79.416936  
7                Forest Hill North, Forest Hill West  43.696948 -79.411307  
8                North Midtown, Yorkville, The Annex  43.672710 -79.405678

We get Central Toronto latitude and longitude values in order to make a map and better visualize the data.

In [7]:
address = 'Central Toronto, Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


Now we can visualize Central Toronto's map

In [8]:
# create map of Central toronto using latitude and longitude values
map_centoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(centoronto_data['Latitude'], centoronto_data['Longitude'], centoronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_centoronto)  
    
map_centoronto

### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = '3WGRGOOWZOLWB3STK2GX0JZN52C1SRGM0LLJZMSXA5KIICJZ' # your Foursquare ID
CLIENT_SECRET = 'H4G3JTG1PIJPMZVWZYKD4XBNN0KCDLSGSH0YXGBI5OUKG1O5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WGRGOOWZOLWB3STK2GX0JZN52C1SRGM0LLJZMSXA5KIICJZ
CLIENT_SECRET:H4G3JTG1PIJPMZVWZYKD4XBNN0KCDLSGSH0YXGBI5OUKG1O5


## 2.- Explore the first neighbourhood in the dataframe

We explore the first neighbourhood in the dataframe, which is:

In [10]:
centoronto_data.loc[0,'Neighbourhood']

'Lawrence Park'

Get the neighbourhoods latitude and longitude values

In [11]:
neigh_latitude = centoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neigh_longitude = centoronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neigh_name = centoronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_latitude, 
                                                               neigh_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Get the top 100 venues that are in Lawrence Park within a radius of 500 meters.

In [12]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,neigh_latitude,neigh_longitude,radius,LIMIT)


In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cabd506f594df4b04d3d2e3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

#### We use the following function to extract the category of each venue.

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Here we extract the relevant information and we put it in a pandas dataframe.

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name   categories        lat        lng
0            Lawrence Park Ravine         Park  43.726963 -79.394382
1              Zodiac Swim School  Swim School  43.728532 -79.382860
2  TTC Bus #162 - Lawrence-Donway     Bus Line  43.728026 -79.382805

In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


### Explore Neighbourhoods in Central Toronto

The following function is used to do the same process for all of Central Toronto's neighbourhoods 

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We run the function:

In [18]:
centraltoronto_venues = getNearbyVenues(names=centoronto_data['Neighbourhood'],
                                   latitudes=centoronto_data['Latitude'],
                                   longitudes=centoronto_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Forest Hill SE, Deer Park, Summerhill West, South Hill, Rathnelly
Roselawn
Forest Hill North, Forest Hill West
North Midtown, Yorkville, The Annex


Here we can see the obtained results:

In [19]:
print(centraltoronto_venues.shape)
centraltoronto_venues.head()

(119, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0     Lawrence Park              43.728020              -79.388790   
1     Lawrence Park              43.728020              -79.388790   
2     Lawrence Park              43.728020              -79.388790   
3  Davisville North              43.712751              -79.390197   
4  Davisville North              43.712751              -79.390197   

                            Venue  Venue Latitude  Venue Longitude  \
0            Lawrence Park Ravine       43.726963       -79.394382   
1              Zodiac Swim School       43.728532       -79.382860   
2  TTC Bus #162 - Lawrence-Donway       43.728026       -79.382805   
3                   Sherwood Park       43.716551       -79.387776   
4         Summerhill Market North       43.715499       -79.392881   

      Venue Category  
0               Park  
1        Swim School  
2           Bus Line  
3               Park  
4  Food & Drink Shop

And here we can see the number of venues for each Neighbourhood

In [20]:
centraltoronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Davisville                                                             37   
Davisville North                                                       11   
Forest Hill North, Forest Hill West                                     4   
Forest Hill SE, Deer Park, Summerhill West, Sou...                     15   
Lawrence Park                                                           3   
North Midtown, Yorkville, The Annex                                    25   
North Toronto West                                                     19   
Roselawn                                                                1   
Summerhill East, Moore Park                                             4   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Davisville                                                              37   
Davisville North                                                        11   
Forest Hill North, Forest Hill West                                      4   
Forest Hill SE, Deer Park, Summerhill West, Sou...                      15   
Lawrence Park                                                            3   
North Midtown, Yorkville, The Annex                                     25   
North Toronto West                                                      19   
Roselawn                                                                 1   
Summerhill East, Moore Park                                              4   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Davisville                                             37              37   
Davisville North                                       11              11   
Forest Hill North, Forest Hill West                     4               4   
Forest Hill SE, Deer Park, Summerhill West, Sou...     15              15   
Lawrence Park                                           3               3   
North Midtown, Yorkville, The Annex                    25              25   
North Toronto West                                     19              19   
Roselawn                                                1               1   
Summerhill East, Moore Park                             4               4   

                                                    Venue Longitude  \
Neighborhood                                                          
Davisville                                                       37   
Davisville North                                                 11   
Forest Hill North, Forest Hill West                               4   
Forest Hill SE, Deer Park, Summerhill West, Sou...               15   
Lawrence Park                                                     3   
North Midtown, Yorkville, The Annex                              25   
North Toronto West                                               19   
Roselawn                                                          1   
Summerhill East, Moore Park                                       4   

                                                    Venue Category  
Neighborhood                                                        
Davisville                                                      37  
Davisville North                                                11  
Forest Hill North, Forest Hill West                              4  
Forest Hill SE, Deer Park, Summerhill West, Sou...              15  
Lawrence Park                                                    3  
North Midtown, Yorkville, The Annex                             25  
North Toronto West                                              19  
Roselawn                                                         1  
Summerhill East, Moore Pa

In [21]:
print('There are {} uniques categories.'.format(len(centraltoronto_venues['Venue Category'].unique())))

There are 63 uniques categories.


## 3.- Analyze Each Neighbourhood

We do one hot encoding the indicate the presence of specific venue in an specific neighbourhood

In [22]:
# one hot encoding
centraltoronto_onehot = pd.get_dummies(centraltoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centraltoronto_onehot['Neighborhood'] = centraltoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [centraltoronto_onehot.columns[-1]] + list(centraltoronto_onehot.columns[:-1])
centraltoronto_onehot = centraltoronto_onehot[fixed_columns]

centraltoronto_onehot.head()

Neighborhood  American Restaurant  Asian Restaurant  BBQ Joint  \
0     Lawrence Park                    0                 0          0   
1     Lawrence Park                    0                 0          0   
2     Lawrence Park                    0                 0          0   
3  Davisville North                    0                 0          0   
4  Davisville North                    0                 0          0   

   Bagel Shop  Breakfast Spot  Brewery  Burger Joint  Bus Line  Café  \
0           0               0        0             0         0     0   
1           0               0        0             0         0     0   
2           0               0        0             0         1     0   
3           0               0        0             0         0     0   
4           0               0        0             0         0     0   

   Cheese Shop  Chinese Restaurant  Clothing Store  Coffee Shop  \
0            0                   0               0            0   
1            0                   0               0            0   
2            0                   0               0            0   
3            0                   0               0            0   
4            0                   0               0            0   

   Convenience Store  Cosmetics Shop  Dance Studio  Dessert Shop  Diner  \
0                  0               0             0             0      0   
1                  0               0             0             0      0   
2                  0               0             0             0      0   
3                  0               0             0             0      0   
4                  0               0             0             0      0   

   Farmers Market  Fast Food Restaurant  Flower Shop  Food & Drink Shop  \
0               0                     0            0                  0   
1               0                     0            0                  0   
2               0                     0            0                  0   
3               0                     0            0                  0   
4               0                     0            0                  1   

   Fried Chicken Joint  Garden  Gift Shop  Gourmet Shop  Greek Restaurant  \
0                    0       0          0             0                 0   
1                    0       0          0             0                 0   
2                    0       0          0             0                 0   
3                    0       0          0             0                 0   
4                    0       0          0             0                 0   

   Gym  History Museum  Hotel  Indian Restaurant  Italian Restaurant  \
0    0               0      0                  0                   0   
1    0               0      0                  0                   0   
2    0               0      0                  0                   0   
3    0               0      0                  0                   0   
4    0               0      0                  0                   0   

   Jewelry Store  Jewish Restaurant  Light Rail Station  Liquor Store  \
0              0                  0                   0             0   
1              0                  0                   0             0   
2              0                  0                   0             0   
3              0                  0                   0             0   
4              0                  0                   0             0   

   Medical Center  Metro Station  Mexican Restaurant  Park  Pet Store  \
0               0              0                   0     1          0   
1               0              0                   0     0          0   
2               0              0                   0     0          0   
3               0              0                   0     1          0   
4               0              0                   0     0          0   

   Pharmacy  Pizza Place  Playground  Pub  Rental Car Lo

In [23]:
centraltoronto_onehot.shape

(119, 64)

#### Grouping rows by neighbourhood and by taking the mean of the frequencey of ocurrence of each category

In [24]:
centraltoronto_grouped = centraltoronto_onehot.groupby('Neighborhood').mean().reset_index()
centraltoronto_grouped

Neighborhood  American Restaurant  \
0                                         Davisville             0.000000   
1                                   Davisville North             0.000000   
2                Forest Hill North, Forest Hill West             0.000000   
3  Forest Hill SE, Deer Park, Summerhill West, So...             0.066667   
4                                      Lawrence Park             0.000000   
5                North Midtown, Yorkville, The Annex             0.040000   
6                                 North Toronto West             0.000000   
7                                           Roselawn             0.000000   
8                        Summerhill East, Moore Park             0.000000   

   Asian Restaurant  BBQ Joint  Bagel Shop  Breakfast Spot   Brewery  \
0          0.027027       0.00    0.000000        0.000000  0.027027   
1          0.090909       0.00    0.000000        0.090909  0.000000   
2          0.000000       0.00    0.000000        0.000000  0.000000   
3          0.000000       0.00    0.066667        0.000000  0.000000   
4          0.000000       0.00    0.000000        0.000000  0.000000   
5          0.000000       0.04    0.000000        0.000000  0.000000   
6          0.000000       0.00    0.000000        0.000000  0.000000   
7          0.000000       0.00    0.000000        0.000000  0.000000   
8          0.000000       0.00    0.000000        0.000000  0.000000   

   Burger Joint  Bus Line      Café  Cheese Shop  Chinese Restaurant  \
0      0.027027  0.000000  0.054054         0.00            0.000000   
1      0.090909  0.000000  0.000000         0.00            0.000000   
2      0.000000  0.000000  0.000000         0.00            0.000000   
3      0.000000  0.000000  0.000000         0.00            0.000000   
4      0.000000  0.333333  0.000000         0.00            0.000000   
5      0.040000  0.000000  0.120000         0.04            0.000000   
6      0.000000  0.000000  0.000000         0.00            0.052632   
7      0.000000  0.000000  0.000000         0.00            0.000000   
8      0.000000  0.000000  0.000000         0.00            0.000000   

   Clothing Store  Coffee Shop  Convenience Store  Cosmetics Shop  \
0        0.000000     0.054054           0.000000            0.00   
1        0.090909     0.000000           0.000000            0.00   
2        0.000000     0.000000           0.000000            0.00   
3        0.000000     0.133333           0.066667            0.00   
4        0.000000     0.000000           0.000000            0.00   
5        0.000000     0.120000           0.040000            0.04   
6        0.052632     0.105263           0.000000            0.00   
7        0.000000     0.000000           0.000000            0.00   
8        0.000000     0.000000           0.000000            0.00   

   Dance Studio  Dessert Shop     Diner  Farmers Market  Fast Food Restaurant  \
0      0.027027      0.081081  0.027027        0.027027              0.000000   
1      0.090909      0.000000  0.000000        0.000000              0.000000   
2      0.000000      0.000000  0.000000        0.000000              0.000000   
3      0.000000      0.000000  0.000000        0.000000              0.000000   
4      0.000000      0.000000  0.000000        0.000000              0.000000   
5      0.000000      0.000000  0.000000        0.000000              0.000000   
6      0.000000      0.052632  0.052632        0.000000              0.052632   
7      0.000000      0.000000  0.000000        0.000000              0.000000   
8      0.000000      0.000000  0.000000        0.000000              0.000000   

   Flower Shop  Food & Drink Shop  Fried Chicken Joint  Garden  Gift Shop  \
0     0.027027           0.000000             0.027027     0.0   0.000000   
1     0.000000           0.090909             0.000000     0.0   0.000000   
2     0.000000           0.000000             0.000000     0.0   0.000000   
3     0.000000   

In [25]:
centraltoronto_grouped.shape

(9, 64)

#### Print each neighbourhood with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in centraltoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = centraltoronto_grouped[centraltoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0  Sandwich Place  0.08
1     Pizza Place  0.08
2    Dessert Shop  0.08
3            Café  0.05
4      Restaurant  0.05


----Davisville North----
               venue  freq
0                Gym  0.09
1              Hotel  0.09
2  Food & Drink Shop  0.09
3       Dance Studio  0.09
4        Pizza Place  0.09


----Forest Hill North, Forest Hill West----
              venue  freq
0              Park  0.25
1  Sushi Restaurant  0.25
2     Jewelry Store  0.25
3             Trail  0.25
4               Pub  0.00


----Forest Hill SE, Deer Park, Summerhill West, South Hill, Rathnelly----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07
3          Supermarket  0.07
4   Light Rail Station  0.07


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1          Swim School  0.33
2             Bus Line  0.33
3  Rental Car Location  0.00
4                  Pub  0.0

#### Put the data inside a pandas dataframe

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = centraltoronto_grouped['Neighborhood']

for ind in np.arange(centraltoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(centraltoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                         Davisville          Dessert Shop   
1                                   Davisville North        Sandwich Place   
2                Forest Hill North, Forest Hill West      Sushi Restaurant   
3  Forest Hill SE, Deer Park, Summerhill West, So...                   Pub   
4                                      Lawrence Park              Bus Line   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Sandwich Place           Pizza Place    Italian Restaurant   
1      Asian Restaurant                   Gym        Breakfast Spot   
2                 Trail         Jewelry Store                  Park   
3           Coffee Shop   American Restaurant            Sports Bar   
4                  Park           Swim School           Yoga Studio   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0             Restaurant                  Café           Coffee Shop   
1           Dance Studio          Burger Joint                 Hotel   
2      Food & Drink Shop                 Diner        Farmers Market   
3  Vietnamese Restaurant   Fried Chicken Joint    Light Rail Station   
4                  Diner        Farmers Market  Fast Food Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0      Sushi Restaurant                  Park           Dance Studio  
1                  Park           Pizza Place      Food & Drink Shop  
2  Fast Food Restaurant           Flower Shop            Yoga Studio  
3        Medical Center           Pizza Place      Convenience Store  
4           Flower Shop     Food & Drink Shop    Fried Chicken Joint

## 4.- Cluster Neighbourhoods

In [29]:
# set number of clusters
kclusters = 3

centraltoronto_grouped_clustering = centraltoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(centraltoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 1, 0])

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

centraltoronto_merged = centoronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
centraltoronto_merged = centraltoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

centraltoronto_merged.head() # check the last columns!

Postal Code          Borough                Neighbourhood   Latitude  \
0         M4N  Central Toronto                Lawrence Park  43.728020   
1         M4P  Central Toronto             Davisville North  43.712751   
2         M4R  Central Toronto           North Toronto West  43.715383   
3         M4S  Central Toronto                   Davisville  43.704324   
4         M4T  Central Toronto  Summerhill East, Moore Park  43.689574   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.388790               2              Bus Line                  Park   
1 -79.390197               0        Sandwich Place      Asian Restaurant   
2 -79.405678               0           Coffee Shop   Sporting Goods Shop   
3 -79.388790               0          Dessert Shop        Sandwich Place   
4 -79.383160               0            Restaurant                   Gym   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Swim School           Yoga Studio                 Diner   
1                   Gym        Breakfast Spot          Dance Studio   
2        Sandwich Place             Gift Shop  Fast Food Restaurant   
3           Pizza Place    Italian Restaurant            Restaurant   
4            Playground          Tennis Court           Flower Shop   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0        Farmers Market  Fast Food Restaurant           Flower Shop   
1          Burger Joint                 Hotel                  Park   
2                 Diner          Dessert Shop         Metro Station   
3                  Café           Coffee Shop      Sushi Restaurant   
4          Dessert Shop                 Diner        Farmers Market   

  9th Most Common Venue 10th Most Common Venue  
0     Food & Drink Shop    Fried Chicken Joint  
1           Pizza Place      Food & Drink Shop  
2    Mexican Restaurant                   Park  
3                  Park           Dance Studio  
4  Fast Food Restaurant    Fried Chicken Joint

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(centraltoronto_merged['Latitude'], centraltoronto_merged['Longitude'], centraltoronto_merged['Neighbourhood'], centraltoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 0

In [32]:
centraltoronto_merged.loc[centraltoronto_merged['Cluster Labels'] == 0, centraltoronto_merged.columns[[1] + list(range(5, centraltoronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
1  Central Toronto               0        Sandwich Place   
2  Central Toronto               0           Coffee Shop   
3  Central Toronto               0          Dessert Shop   
4  Central Toronto               0            Restaurant   
5  Central Toronto               0                   Pub   
7  Central Toronto               0      Sushi Restaurant   
8  Central Toronto               0        Sandwich Place   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1      Asian Restaurant                   Gym        Breakfast Spot   
2   Sporting Goods Shop        Sandwich Place             Gift Shop   
3        Sandwich Place           Pizza Place    Italian Restaurant   
4                   Gym            Playground          Tennis Court   
5           Coffee Shop   American Restaurant            Sports Bar   
7                 Trail         Jewelry Store                  Park   
8                  Café           Coffee Shop           Pizza Place   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
1           Dance Studio          Burger Joint                 Hotel   
2   Fast Food Restaurant                 Diner          Dessert Shop   
3             Restaurant                  Café           Coffee Shop   
4            Flower Shop          Dessert Shop                 Diner   
5  Vietnamese Restaurant   Fried Chicken Joint    Light Rail Station   
7      Food & Drink Shop                 Diner        Farmers Market   
8    American Restaurant        History Museum     Indian Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
1                  Park           Pizza Place      Food & Drink Shop  
2         Metro Station    Mexican Restaurant                   Park  
3      Sushi Restaurant                  Park           Dance Studio  
4        Farmers Market  Fast Food Restaurant    Fried Chicken Joint  
5        Medical Center           Pizza Place      Convenience Store  
7  Fast Food Restaurant           Flower Shop            Yoga Studio  
8     Jewish Restaurant          Liquor Store                   Park

Cluster 0 or RED cluster is obviously a *'FOOD AND DRINKS'* cluster.

#### Cluster 1

In [33]:
centraltoronto_merged.loc[centraltoronto_merged['Cluster Labels'] == 1, centraltoronto_merged.columns[[1] + list(range(5, centraltoronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
6  Central Toronto               1                Garden   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
6           Yoga Studio          Dance Studio        History Museum   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
6                   Gym      Greek Restaurant          Gourmet Shop   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
6             Gift Shop   Fried Chicken Joint      Food & Drink Shop

Cluster 1 or PURPLE cluster includes *'CULTURAL'* venues.

#### Cluster 2

In [34]:
centraltoronto_merged.loc[centraltoronto_merged['Cluster Labels'] == 2, centraltoronto_merged.columns[[1] + list(range(5, centraltoronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0  Central Toronto               2              Bus Line   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Park           Swim School           Yoga Studio   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                 Diner        Farmers Market  Fast Food Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Flower Shop     Food & Drink Shop    Fried Chicken Joint

Cluster 2 or GREEN cluster includes *'PUBLIC SPACES'* venues.